In [1]:
import os
import re
import traceback

In [2]:
import Levenshtein
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pickle
import time
import numpy as np
import gc

In [3]:
from unitils import from_entry,match_type_cur,get_entry,from_value,jaccard,hint,local_les,get_type

In [4]:
import unitils

In [5]:
from unitils import get_ans,ans_clean,pre_clean,f1
from one_jump import one_jump_solver
from two_jump import two_jump_solver,down_to_low
from config import df,stop_words

## 同义词表

In [6]:
SYNONYM_MAP = {
    '丈夫': '丈夫配偶',
    '老公': '丈夫配偶',
    '妻子': '妻子配偶',
    '夫人': '夫人妻子配偶',
    '媳妇': '妻子配偶',
    '老家': '老家籍贯',
    '故乡': '故乡籍贯',
    '家乡': '家乡籍贯', 
    '英文':'外文英文',
    '西班牙语':'外文西班牙语',
    '贡献' : '贡献主要成就',
    '功绩' : '功绩主要成就',
    '丰功伟绩' : '丰功伟绩主要成就',
    '来源' : '来源出处',
    '在金庸小说《天龙八部》中':''
}

In [7]:
SAMPLE_LENGTH = 20

# 单解器

## one_jump_feature

In [8]:
def one_jump_feature(one_jump_feature_ans,one_jump_feature_ss,one_jump_feature_label = []):
    
    """
    raw  value 定义为 (roots,edge,ans_node)
    """
    
    anses = []
    tmp_x = []
    one_jump_raw = {}
    
    fff,raw = one_jump_solver(one_jump_feature_ans,one_jump_feature_ss)
    
    for key in fff:
        anses.append(key)
        tmp_x.append(fff[key])

    try:
        tmp_x = preprocessing.scale(tmp_x)

        one_log_y = one_jump_clf.predict_proba(tmp_x)[:,1]
        one_mlp_y = one_MLP.predict(tmp_x).reshape(-1)
        one_mlp_y = one_mlp_y/one_mlp_y.max()

        y = 0.85*one_log_y + 0.15*one_mlp_y      
        one_jump_score = y.max()
        one_jump_pre = [x for x,z in zip(anses,y) if z == one_jump_score]
        for high in one_jump_pre:
            one_jump_raw[high] = raw[high]
        one_jump_vec = fff[one_jump_pre[0]]        
        if one_jump_feature_label:
            one_jump_f1 = f1(pre_clean(one_jump_pre),pre_clean(one_jump_feature_label))
        else:
            one_jump_f1 = 0
    except ValueError:
        one_jump_pre = []
        one_jump_f1 = 0
        one_jump_vec = np.zeros(7)
        one_jump_score = 0
    
    return one_jump_pre,one_jump_raw,one_jump_f1,one_jump_vec,one_jump_score

## two_jump_feature

In [9]:
def two_jump_feature(two_jump_feature_ans,two_jump_feature_ss,two_jump_feature_label = []):
    
    """
    raw value 定义为 (roots,first_edge,brige_node,second_edge,ans_node) 
    """
    
    anses = []
    tmp_x = []
    two_jump_raw = {}

    fff,raw = two_jump_solver(two_jump_feature_ans,two_jump_feature_ss)
    aaa = down_to_low(fff)
    for key in aaa:
        anses.append(key)
        tmp_x.append(fff[key])

    try:
        tmp_x = preprocessing.scale(tmp_x)

        y = two_jump_clf.predict_proba(tmp_x)
        
        two_jump_score = y[:,1].max()
        
        two_jump_pre = [x for x,z in zip(anses,y[:,1]) if z == two_jump_score]
        for high in two_jump_pre:
            two_jump_raw[high] = raw[high]
        
        two_jump_vec = fff[two_jump_pre[0]]

        #two_jump_pre = pre_clean(two_jump_pre)
        
        if two_jump_feature_label:
            two_jump_f1 = f1(pre_clean(two_jump_pre),pre_clean(two_jump_feature_label))
        else:
            two_jump_f1 = 0

        

    except ValueError:
        two_jump_pre = []
        two_jump_f1 = 0
        two_jump_vec = np.zeros(7)
        two_jump_score = 0
    
    return two_jump_pre,two_jump_raw,two_jump_f1,two_jump_vec,two_jump_score

# 读取预解空间

In [11]:
def save_to_disk(var):
    
    import pickle
    import inspect
    #var_name = list(dict(var = var).keys())[0]
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    var_name = [var_name for var_name, var_val in callers_local_vars if var_val is var][0]
    
    time_id = time.strftime("_%m_%d_%H_%M", time.localtime())
    
    with open(var_name + time_id + ".bin","wb") as f:
        pickle.dump(var,f)
    print(var_name + time_id + ".bin")

In [23]:
#save_to_disk(ans_cache)

ans_cache_07_14_12_52.bin


In [ ]:
with open("ans_cache_07_14_12_52.bin","rb") as f:
    ans_cache = pickle.load(f)

# 单类型求解器

## 单跳问题

### 训练单跳问题模型

In [22]:
one_pos = []
for q_index in range(1283):
    ss = df["question"][q_index]
    label = ans_clean(df["answer"][q_index])
    ans = ans_cache[q_index][1]
    fff,raw =  one_jump_solver(ans,ss)    
    for key,value in fff.items():
        if set(pre_clean(key)).intersection(label):
            one_pos.append(q_index)
            print(q_index)
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
two_test = """523 530 532 535 542 549 552 553 554 556 558 559 561 565 569 573 574 584 594 595 596 598 599\
 1019 1023 1028 1029 1033 1034 1036 1038 1039 1043 1049 1050 1051 1052 1054\
 1057 1059 1061 1062 1071 1078 1086 1088 1090 1095 1096 1099 1101 1102 1103 1108 1109 1111 1112 1115 1118 1122\
 1123 1124 1125 1129 1130 1132 1135 1138 1140 1143 1145 1149"""

In [25]:
final_one_pos = set(one_pos) - set([int(x) for x in two_test.split(" ")])

In [26]:
one_Y = []
one_X = []
for q_index in final_one_pos:
    
    print(q_index)    
    ss = df["question"][q_index]
    label = ans_clean(df["answer"][q_index])
    
    ans = ans_cache[q_index][1]
    fff,raw =  one_jump_solver(ans,ss)
    
    for key,value in fff.items():
        if pre_clean(key)[0] in label:
            
            one_pos.append(q_index)
            one_Y.append(1)
        else:
            one_Y.append(0)
        one_X.append(value)

0
1
2
3
4
5
6
8
9
10
11
12
14
15
16
17
18
19
20
21
22
24
25
26
27
28
29
30
31
32
33
34
35
37
38
39
40
41
45
46
47
48
49
50
51
52
53
54
55
56
57
58
60
61
62
63
64
65
66
67
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
87
89
90
91
92
94
95
96
97
99
100
101
102
103
104
105
107
108
109
110
111
112
114
115
116
117
118
119
121
122
123
124
125
126
127
128
129
130
131
133
134
135
136
137
138
139
140
141
142
143
145
146
148
149
152
153
154
155
156
158
159
160
161
162
163
164
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
194
195
196
197
199
200
201
202
203
204
205
206
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
237
238
240
241
243
244
245
246
247
248
249
250
251
253
254
255
256
257
258
259
260
262
263
264
265
266
267
268
269
270
271
272
274
275
276
277
278
279
280
281
283
284
285
287
288
290
291
293
295
297
298
299
300
301
302
303
304
306
307
308
309
310
311
313
315
316
317

In [29]:
log_x = preprocessing.scale(one_X)
X_train, X_test, y_train, y_test = train_test_split(log_x,one_Y, test_size=0.382, random_state=4)
from sklearn.linear_model import LogisticRegressionCV
one_jump_clf = LogisticRegressionCV(penalty= "l2",class_weight='balanced')
#two_jump_clf.fit(X_train,y_train)
one_jump_clf.fit(log_x,one_Y)
one_jump_clf.score(X_test,y_test)

0.8500693295164825

In [1688]:
save_to_disk(one_jump_clf)

one_jump_clf_07_20_12_41.bin


In [211]:
from keras.layers import Input, Dense, GRU, Dropout,BatchNormalization
from keras.models import Model

def MLP(optimizers="adam"):
    """ 
    Create and return a keras model of a MLP 
    """

    inputs = Input(shape=(4,))
    dense_1 = Dense(8, activation='relu')(inputs)
    dense_2 = Dense(3, activation='relu')(dense_1)
    dropout = Dropout(0.2)(dense_2)
    outputs = Dense(1,activation='sigmoid')(dropout)
#8 3 1
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        #loss='mse',
        optimizer=optimizers,
        metrics=['mse'],
    )

    return model

In [425]:
one_MLP = MLP()

In [426]:
one_MLP.fit(log_x,np.array(one_Y),epochs= 10 ,validation_split= 0.1)

Train on 52673 samples, validate on 5853 samples
Epoch 1/10
52673/52673 [==============================] - 14s 261us/step - loss: 0.1448 - mean_squared_error: 0.0383 - val_loss: 0.2050 - val_mean_squared_error: 0.0635
Epoch 2/10
52673/52673 [==============================] - 11s 204us/step - loss: 0.0965 - mean_squared_error: 0.0241 - val_loss: 0.2132 - val_mean_squared_error: 0.0663
Epoch 3/10
52673/52673 [==============================] - 10s 186us/step - loss: 0.0929 - mean_squared_error: 0.0231 - val_loss: 0.2213 - val_mean_squared_error: 0.0694
Epoch 4/10
52673/52673 [==============================] - 10s 183us/step - loss: 0.0904 - mean_squared_error: 0.0226 - val_loss: 0.2240 - val_mean_squared_error: 0.0696
Epoch 5/10
52673/52673 [==============================] - 11s 205us/step - loss: 0.0886 - mean_squared_error: 0.0222 - val_loss: 0.2329 - val_mean_squared_error: 0.0722
Epoch 6/10
52673/52673 [==============================] - 11s 212us/step - loss: 0.0872 - mean_squared_err

In [779]:
one_MLP.save("final_one_MLP_new.bin")

## 双跳问题

### 挑选双跳可解决的问题

In [17]:
two_jump_pos = []
for q_index in range(1283):
    q_index = int(q_index)
    ss = df["question"][q_index]
    label = ans_clean(df["answer"][q_index])
    print("-"*130)
    print(q_index,ss)
    ans = ans_cache[q_index][1]
    fff ,raw =  two_jump_solver(ans,ss)
    
    # 将预选答案降到100以内,防止样本不均衡
    hhh = down_to_low(fff)
    for key in hhh:
        if pre_clean(key)[0] in label:
            print("+"*130)
            print(key)
            two_jump_pos.append([q_index,key])

----------------------------------------------------------------------------------------------------------------------------------
0 莫妮卡·贝鲁奇的代表作
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
1 《湖上草》是谁的诗
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
2 龙卷风的英文名是什么
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"Tornado"
----------------------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<安徒生童话>
----------------------------------------------------------------------------------------------------------------------------------
26 谁是天蝎座的守护神
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
27 到手香又被称为什么
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<还魂草_（卷柏别名）>
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<延命草>
------------------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"希腊获胜"
----------------------------------------------------------------------------------------------------------------------------------
49 新海诚动漫《你的名字》由哪个公司出品
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<东宝株式会社>
----------------------------------------------------------------------------------------------------------------------------------
50 国际足联世界杯的冠军奖杯叫做什么
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"雷米特杯"
++++++++++++++++++

----------------------------------------------------------------------------------------------------------------------------------
71 古希腊的首都是什么地方
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<雅典_（希腊首都）>
----------------------------------------------------------------------------------------------------------------------------------
72 “生命在于运动”是谁提出的
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<伏尔泰_（法国思想家、文学家、哲学家）>
----------------------------------------------------------------------------------------------------------------------------------
73 北邮现在的校长是谁
++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<电_（自然现象之一）>
----------------------------------------------------------------------------------------------------------------------------------
93 多人在线战术竞技游戏有哪些代表作
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
94 腾讯一共有多少员工
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
95 杨红樱什么时候创作的淘气包马小跳
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++

...

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<天衣有风>
----------------------------------------------------------------------------------------------------------------------------------
1178 香奈儿五号香水是什么时候发行的
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"1921"
----------------------------------------------------------------------------------------------------------------------------------
1179 孟京辉导演廖一梅编剧的作品是
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<恋爱的犀牛>
--------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<亚热带季风气候>
----------------------------------------------------------------------------------------------------------------------------------
1200 《根鸟》的作者毕业于哪个大学
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<北京大学>
----------------------------------------------------------------------------------------------------------------------------------
1201 由卡尔·马克思发起的思想潮流，其著名口号是?
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"“全世界无产者，联合起来！”"
-

----------------------------------------------------------------------------------------------------------------------------------
1222 卖报歌的谱曲者祖籍在哪里
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<玉溪_（云南省地级市）>
----------------------------------------------------------------------------------------------------------------------------------
1223 俄罗斯的首都是何时建城的
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"1147"
----------------------------------------------------------------------------------------------------------------------------------
1224 电影《白日焰火》的导演有哪些代表作品
++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<演员_（职业）>
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<歌手_（职业名称）>
----------------------------------------------------------------------------------------------------------------------------------
1243 张一山和杨紫合作过的作品有哪些
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
1244 那位普瓦捷大学毕业的法国著名哲学家做出了什么突出贡献
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
1245 凯撒的

----------------------------------------------------------------------------------------------------------------------------------
1266 基因载体的组成是什么
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------------------------------------------------------------------------
1267 加拿大的国花长度为多少
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"13厘米"
----------------------------------------------------------------------------------------------------------------------------------
1268 和钢铁侠有相同能力的人都有谁
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-----------------------

### 训练样本生成

In [45]:
two_X = []
two_Y = []
for q_index in set([ x for x,y in two_jump_pos]):
    #q_index = 552
    tmp_x = []
    ss = df["question"][q_index]    
    label = ans_clean(df["answer"][q_index])
    ans = ans_cache[q_index][1]
    print("-"*130)
    print(q_index,ss)
    fff,raw = two_jump_solver(ans,ss)
    # 防止样本过于不均衡
    aaa = down_to_low(fff)
    
    for key in aaa:
        if pre_clean(key)[0] in label:
            two_Y.append(1)
        else:
            two_Y.append(0)
        tmp_x.append(fff[key])
    
    two_X.extend(tmp_x)

----------------------------------------------------------------------------------------------------------------------------------
2 龙卷风的英文名是什么
----------------------------------------------------------------------------------------------------------------------------------
5 叔本华信仰什么宗教
----------------------------------------------------------------------------------------------------------------------------------
6 大兴安岭的终点是哪里
----------------------------------------------------------------------------------------------------------------------------------
7 演员梅艳芳有多高
----------------------------------------------------------------------------------------------------------------------------------
11 叶文洁毕业于哪个大学
----------------------------------------------------------------------------------------------------------------------------------
14 清明节起源于哪里
----------------------------------------------------------------------------------------------------------------------------------
19 清朝词人纳

----------------------------------------------------------------------------------------------------------------------------------
124 仙剑奇侠传五前传是哪个系列的游戏
----------------------------------------------------------------------------------------------------------------------------------
125 “共产主义五大人物”都有哪些
----------------------------------------------------------------------------------------------------------------------------------
126 《胡笳十八拍》是谁的传世之作
----------------------------------------------------------------------------------------------------------------------------------
127 悉尼有哪些火车站
----------------------------------------------------------------------------------------------------------------------------------
128 中秋节都有哪些习俗
----------------------------------------------------------------------------------------------------------------------------------
129 鸿上光生是哪部作品中的人物
--------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
232 《星球大战》系列电影的发行公司是哪个
----------------------------------------------------------------------------------------------------------------------------------
233 冰心原名是什么
----------------------------------------------------------------------------------------------------------------------------------
234 漫水河流经的面积有多大
----------------------------------------------------------------------------------------------------------------------------------
237 王力宏的英文名是什么
----------------------------------------------------------------------------------------------------------------------------------
238 日本间谍川岛芳子是哪个民族的
----------------------------------------------------------------------------------------------------------------------------------
239 吉林大学校歌是
---------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
362 与会计相关的证书都有什么
----------------------------------------------------------------------------------------------------------------------------------
363 恐龙中的壮龙亚科存活于哪个时期
----------------------------------------------------------------------------------------------------------------------------------
364 喜乐庄村位于海拔多少米的地方
----------------------------------------------------------------------------------------------------------------------------------
368 化学式为"C6H12O6"的物质有哪些
----------------------------------------------------------------------------------------------------------------------------------
369 北科归哪个国务院部门管
----------------------------------------------------------------------------------------------------------------------------------
372 足球加入奥运会是在哪一年
-----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
483 四川大学的主要奖项有哪些
----------------------------------------------------------------------------------------------------------------------------------
484 韩国属于哪个大洲
----------------------------------------------------------------------------------------------------------------------------------
485 亚马逊公司成立于哪一年
----------------------------------------------------------------------------------------------------------------------------------
486 网络剧《毛骗》在那个网站平台上播出
----------------------------------------------------------------------------------------------------------------------------------
487 音乐人刘洲喜欢哪些动物
----------------------------------------------------------------------------------------------------------------------------------
489 综艺节目康熙来了是谁主持的
----------------------------------------------------------------------------------------------------------------

...

----------------------------------------------------------------------------------------------------------------------------------
835 《丰乳肥臀》的作者的别名有
----------------------------------------------------------------------------------------------------------------------------------
839 冥王星的发现者是哪国人
----------------------------------------------------------------------------------------------------------------------------------
841 拉丁学名为"Gerbera kunzeana A. Br. et Aschers."的植物属于哪一个纲
----------------------------------------------------------------------------------------------------------------------------------
842 小说《伊豆的舞女》的作者出生日期是哪年? 
----------------------------------------------------------------------------------------------------------------------------------
843 叔本华信仰的宗教主要分布在哪些区域
----------------------------------------------------------------------------------------------------------------------------------
844 冰与火之歌拍摄地国土面积有多大
--------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
950 出生于-413年，逝世于-354年的那位琴师有什么代表作品
----------------------------------------------------------------------------------------------------------------------------------
951 中国都有哪些网球基地
----------------------------------------------------------------------------------------------------------------------------------
952 唐人公司出品的射雕英雄传是哪一年的版本
----------------------------------------------------------------------------------------------------------------------------------
955 命运交响曲的作曲是谁
----------------------------------------------------------------------------------------------------------------------------------
956 习大大的信仰是什么
----------------------------------------------------------------------------------------------------------------------------------
960 释永信担任主持的寺庙有哪些著名景点
------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1062 《论衡》的作者是哪个民族的人氏
----------------------------------------------------------------------------------------------------------------------------------
1066 第43届奥斯卡金像奖最佳影片的导演是谁?
----------------------------------------------------------------------------------------------------------------------------------
1068 中国首都的电话区号是多少
----------------------------------------------------------------------------------------------------------------------------------
1069 有一部电影是由马里奥·普佐的小说改编，马龙·白兰度主演，请问它在哪一年上映
----------------------------------------------------------------------------------------------------------------------------------
1070 前妻是葛天的110米栏世界冠军，他的教练是谁
----------------------------------------------------------------------------------------------------------------------------------
1071 《野草》的作者的代表作品有哪些
---------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1162 小仲马的代表作品是什么
----------------------------------------------------------------------------------------------------------------------------------
1163 画出《墨虾》的画家是哪里人
----------------------------------------------------------------------------------------------------------------------------------
1171 杨紫琼、章子怡合作过哪些作品
----------------------------------------------------------------------------------------------------------------------------------
1172 日本哪座城市以紫罗兰为市花
----------------------------------------------------------------------------------------------------------------------------------
1173 飞行棋有几种颜色
----------------------------------------------------------------------------------------------------------------------------------
1175 1997版泰坦尼克号的主演都有谁?
------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1270 四书五经都包括什么
----------------------------------------------------------------------------------------------------------------------------------
1272 直升机有哪些应用
----------------------------------------------------------------------------------------------------------------------------------
1273 足球运动员梅西所在的球队现任主教练是谁
----------------------------------------------------------------------------------------------------------------------------------
1276 陵寝是惠陵的人的主要成就是
----------------------------------------------------------------------------------------------------------------------------------
1279 “超人主义”的提出人的代表作品有哪些
----------------------------------------------------------------------------------------------------------------------------------
1280 出版书籍《宋史新探》的出版社是何时成立的
-------------------------------------------------------------------------------------------

In [48]:
two_X = preprocessing.scale(two_X)
X_train, X_test, y_train, y_test = train_test_split(two_X,two_Y, test_size=0.382, random_state=4)
from sklearn.linear_model import LogisticRegressionCV
two_jump_clf = LogisticRegressionCV(penalty="l2")
two_jump_clf.fit(two_X,two_Y)
two_jump_clf.score(X_test,y_test)

0.9678737233054782

In [ ]:
save_to_disk(two_jump_clf)

# 两类答案产生的特征

In [460]:
two_class_data = []

for q_index in range(1283):
    try:
        ans_num = 0
        ss = df["question"][q_index]
        print("-"*130)
        print(q_index,ss)
        label = ans_clean(df["answer"][q_index])
        ans = ans_cache[q_index][1]

        final_feature = []
        final_feature.append(q_index)
        final_feature.append(one_jump_feature(ans,ss,label))
        final_feature.append(two_jump_feature(ans,ss,label))
        
        two_class_data.append(final_feature)
    except Exception as e:
        traceback.print_exc()

----------------------------------------------------------------------------------------------------------------------------------
0 莫妮卡·贝鲁奇的代表作
----------------------------------------------------------------------------------------------------------------------------------
1 《湖上草》是谁的诗
----------------------------------------------------------------------------------------------------------------------------------
2 龙卷风的英文名是什么
----------------------------------------------------------------------------------------------------------------------------------
3 新加坡的水域率是多少
----------------------------------------------------------------------------------------------------------------------------------
4 商朝在哪场战役中走向覆灭
----------------------------------------------------------------------------------------------------------------------------------
5 叔本华信仰什么宗教
----------------------------------------------------------------------------------------------------------------------------------
6 大兴

/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


----------------------------------------------------------------------------------------------------------------------------------
8 被誉为万岛之国的是哪个国家
----------------------------------------------------------------------------------------------------------------------------------
9 北京奥运会的口号是什么
----------------------------------------------------------------------------------------------------------------------------------
10 新加坡的政治元老是谁
----------------------------------------------------------------------------------------------------------------------------------
11 叶文洁毕业于哪个大学
----------------------------------------------------------------------------------------------------------------------------------
12 “两弹一星”指的是什么
----------------------------------------------------------------------------------------------------------------------------------
13 “超级碗”是什么体育项目联赛的年度冠军赛
---------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
64 我们会在哪个民俗节日里猜灯谜
----------------------------------------------------------------------------------------------------------------------------------
65 撒哈拉沙漠的最高温度是
----------------------------------------------------------------------------------------------------------------------------------
66 唐嫣工作室旗下都有哪些艺人
----------------------------------------------------------------------------------------------------------------------------------
67 白居易的那首《长恨歌》一共有多少字
----------------------------------------------------------------------------------------------------------------------------------
68 肯德基的创始人是谁
----------------------------------------------------------------------------------------------------------------------------------
69 《悼李夫人赋》是谁的作品
------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
120 杜鹃鸟被归类为哪个亚门
----------------------------------------------------------------------------------------------------------------------------------
121 斗破苍穹是谁写的
----------------------------------------------------------------------------------------------------------------------------------
122 伊拉克国花是什么
----------------------------------------------------------------------------------------------------------------------------------
123 吉野家创建于什么时候
----------------------------------------------------------------------------------------------------------------------------------
124 仙剑奇侠传五前传是哪个系列的游戏
----------------------------------------------------------------------------------------------------------------------------------
125 “共产主义五大人物”都有哪些
---------------------------------------------------------------------------------------------------------------------

...

----------------------------------------------------------------------------------------------------------------------------------
1056 武汉大学出了哪些导演
----------------------------------------------------------------------------------------------------------------------------------
1057 AlphaGo的设计者是谁
----------------------------------------------------------------------------------------------------------------------------------
1058 太宰治写过什么中篇小说
----------------------------------------------------------------------------------------------------------------------------------
1059 歌曲《红玫瑰》的原唱歌手是什么民族的
----------------------------------------------------------------------------------------------------------------------------------
1060 毕业于哈工大的乒乓球运动员都有谁
----------------------------------------------------------------------------------------------------------------------------------
1061 金正恩父亲的军衔有多高
---------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1110 Steve Jobs什么时候去世
----------------------------------------------------------------------------------------------------------------------------------
1111 南起北山路，北接天目山路的这条路长为多少
----------------------------------------------------------------------------------------------------------------------------------
1112 《悲惨世界》的作者的毕业院校是
----------------------------------------------------------------------------------------------------------------------------------
1113 "2009年第一轮第3位被雷霆队选中"的球员获得了哪些奖项
----------------------------------------------------------------------------------------------------------------------------------
1114 女王尼托克丽丝统治的王国面积有多大
----------------------------------------------------------------------------------------------------------------------------------
1115 《乡土中国》的作者毕业于哪个大学
-----------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1164 秦始皇的继承者的结局
----------------------------------------------------------------------------------------------------------------------------------
1165 清朝有哪些政治家
----------------------------------------------------------------------------------------------------------------------------------
1166 高加索地区所属大陆的人口密度
----------------------------------------------------------------------------------------------------------------------------------
1167 美国总统罗斯福的母校创办时间
----------------------------------------------------------------------------------------------------------------------------------
1168 史诗《伊利亚特》的作者诞生于何时
----------------------------------------------------------------------------------------------------------------------------------
1169 儒家文化对中国影响深远，试着举出几个以它为信仰的哲学家
-----------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1218 《随想录》的作者其原名是
----------------------------------------------------------------------------------------------------------------------------------
1219 《时间简史》的作者是哪个国家的
----------------------------------------------------------------------------------------------------------------------------------
1220 韩愈写过什么论说文
----------------------------------------------------------------------------------------------------------------------------------
1221 小迪是哪个经纪公司的
----------------------------------------------------------------------------------------------------------------------------------
1222 卖报歌的谱曲者祖籍在哪里
----------------------------------------------------------------------------------------------------------------------------------
1223 俄罗斯的首都是何时建城的
------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
1272 直升机有哪些应用
----------------------------------------------------------------------------------------------------------------------------------
1273 足球运动员梅西所在的球队现任主教练是谁
----------------------------------------------------------------------------------------------------------------------------------
1274 环太平洋的导演的出生地是哪
----------------------------------------------------------------------------------------------------------------------------------
1275 有哪些思想家信仰共产主义
----------------------------------------------------------------------------------------------------------------------------------
1276 陵寝是惠陵的人的主要成就是
----------------------------------------------------------------------------------------------------------------------------------
1277 出生于日本毕业于美国的中国建筑师是
------------------------------------------------------------------------------------------------

# 训练CNN相似度模型

## 加载字向量

In [48]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format("file.txt", binary=False,encoding="utf8")

## CNN 用特征

In [469]:
from sklearn.preprocessing import StandardScaler

In [464]:
# 将单类问题生成的特征转化为CNN特征
this_feature = []
for row in two_class_data:
    #print(row)
    
    index,one,two = row
    ss = df["question"][index]
    
    pattern = re.compile("[\"“”‘’''\<\>]")
    pre,path,F1,feature,score = two
    for key in two[1]:
        char = ""
        line = two[1][key]
        prop = line[1] + line[3]
        if len(line) > 5:
            roots = (line[0][0],line[5])
        else:
            roots = (line[0][0])
        char = "".join(roots) + line[1] + line[3]        
        this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))

    pre,path,F1,feature,score = one
    char = ""
    prop = ""
    for key in one[1]:
        path = one[1][key]["path"]
        for line in path:
            tmp = line[0] + line[2]
        if tmp not in char:
            char += tmp 
            prop += line[2] 
            this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))

In [470]:
def seq_to_vec(seq_to_vec_seq,length = SAMPLE_LENGTH):
    x = []
    for row in seq_to_vec_seq[:length]:
        #print(row)
        if row in model:
            x.append(model[row.upper()])

    for i in range(length - len(x)):
        x.append(model["<PAD>"])
    return x

In [467]:
train_pre = []
train_que = []
train_y = []
for row in this_feature:
    pre,prop,que,F1 = row
    
    tmp_pre = np.array(seq_to_vec(pre,SAMPLE_LENGTH))
    tmp_que = np.array(seq_to_vec(que,SAMPLE_LENGTH))
    train_pre.append(tmp_pre)
    train_que.append(tmp_que)
    train_y.append(F1)
train_pre = np.array(train_pre)
train_que = np.array(train_que)

In [471]:
scaler = StandardScaler(copy=False)
tmp_vec = np.concatenate([train_pre,train_que],axis = 0)
t_train_pre = scaler.fit_transform(train_pre.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
t_train_que = scaler.fit_transform(train_que.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
train_x = [t_train_pre,t_train_que] * 5

In [472]:
tmp_vec = np.concatenate([train_pre,train_que],axis = 0)

In [473]:
scaler.fit(tmp_vec.reshape(-1,64))

StandardScaler(copy=False, with_mean=True, with_std=True)

In [474]:
t_train_pre = scaler.transform(train_pre.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)

In [475]:
t_train_que = scaler.transform(train_que.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)

In [477]:
train_x = [t_train_pre,t_train_que] * 5

In [1686]:
save_to_disk(scaler)

train_x_07_20_09_40.bin


### CNN 模型

In [478]:
from keras.layers import Input, Dense, GRU, Dropout, BatchNormalization, \
                         MaxPooling1D, Conv1D, Flatten, Concatenate,AveragePooling1D
from keras.models import Model

# 构建相似度模型 还是 直接 输出 f1值作为

def cnn_sim(embedding_size = 64, output_length = 1,optimizers="adam"):
    """ Create and return a keras model of a CNN """

    NB_FILTER = 256
    NGRAM_LENGTHS = [1, 2, 3 ,4 ,5]

    conv_layers, inputs = [], []
    inputs_2 = []

    for ngram_length in NGRAM_LENGTHS:
        current_input = Input(shape=(SAMPLE_LENGTH, embedding_size))
        current_input_2 = Input(shape=(SAMPLE_LENGTH, embedding_size))
        
        inputs.append(current_input)
        inputs.append(current_input_2)
        conv = Conv1D(
            NB_FILTER,
            ngram_length,
            kernel_initializer='lecun_uniform',
            activation='tanh',
        )
        convolution = conv(current_input)
        convolution_2 = conv(current_input_2)

        pool_size = SAMPLE_LENGTH - ngram_length + 1
        pooling = MaxPooling1D(pool_size=pool_size)(convolution)
        pooling_2 = MaxPooling1D(pool_size=pool_size)(convolution_2)
        
        conv_layers.append(pooling)
        conv_layers.append(pooling_2)
    
    merged = Concatenate()(conv_layers)
    #BN = BatchNormalization()(merged)
    dropout = Dropout(0.5)(merged)
    flattened = Flatten()(dropout)
    
    outputs = Dense(output_length, activation='sigmoid')(flattened)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        #loss = "mse",
        optimizer=optimizers,
        metrics=['mse'],
    )

    return model

In [ ]:
sim = cnn_sim()
sim.fit(x = train_x,y = train_y,epochs = 20,validation_split= 0.1)

In [780]:
sim.save("final_cnn_new.bin")

## CNN特征构造函数

In [64]:
def feature_for_cnn(final_feature,final_feature_scaler = scaler):
    index,one,two = final_feature
    #ss = df["question"][index]
    this_feature = []
    pattern = re.compile("[\"“”‘’''\<\>]")
    
    pre,path,F1,feature,score = two
    for key in two[1]:
        char = ""
        line = two[1][key]
        prop = line[1] + line[3]
        if len(line) > 5:
            roots = (line[0][0],line[5])
        else:
            roots = (line[0][0])
        char = "".join(roots) + line[1] + line[3]        
        this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))

    pre,path,F1,feature,score = one
    char = ""
    prop = ""
    for key in one[1]:
        path = one[1][key]["path"]
        for line in path:
            tmp = line[0] + line[2]
        if tmp not in char:
            char += tmp 
            prop += line[2] 
            this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))
    
    test_pre = []
    test_que = []
    test_ans = []
    for row in this_feature:
        pre,prop,que,key = row

        tmp_pre = np.array(seq_to_vec(pre,SAMPLE_LENGTH))
        tmp_que = np.array(seq_to_vec(que,SAMPLE_LENGTH))
        test_pre.append(tmp_pre)
        test_que.append(tmp_que)
        test_ans.append(key)
    test_pre = np.array(test_pre)
    test_que = np.array(test_que)
    try:
        t_test_pre = final_feature_scaler.transform(test_pre.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
        t_test_que = final_feature_scaler.transform(test_que.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
    
        test_x = [t_test_pre,t_test_que] * 5
    except ValueError:
        return [],[]
    return test_x,test_ans

# 感知机模型

In [480]:
fff_feature = []
fff_y = []
for brige in two_class_data:
    
    tmp_feature = []
    #brige = (num,one,two,mul)
    num,one,two = brige
    #print(num)
    test_x,test_ans = feature_for_cnn(brige)
    
    if not test_x:
        #submit.append([])
        continue
    
    test_y = sim.predict(test_x)[:,0]
    
    try:
        one_pre_score = test_y[[x in one[0] for x in test_ans]].max()
    except ValueError :
        one_pre_score = 0
    try:
        two_pre_score = test_y[[x in two[0] for x in test_ans]].max()
    except ValueError:
        two_pre_score = 0
    #这里一定要回到分类器，不然会丢失很多东西
    # 但也有可能有好处
    tmp_feature.extend((one_pre_score,two_pre_score))
    tmp_feature.extend(one[3])
    tmp_feature.extend(two[3])
    
    tmp_feature.extend((two[4],one[4]))
    
    #fff_y.append(np.array([one[2],two[2],mul[2]]).argmax())
    fff_y.append([one[2],two[2]])
    fff_feature.append(tmp_feature)

In [482]:
scaler_2 = StandardScaler(copy=False)

In [483]:
fff_train_x = scaler_2.fit_transform(np.array(fff_feature))

In [ ]:
save_to_disk(scaler_2)

In [484]:
from keras.layers import Input, Dense, GRU, Dropout,BatchNormalization
from keras.models import Model

def my_model(optimizers="adam"):
    """ Create and return a keras model of a MLP """

    inputs = Input(shape=(15,))
    dense_1 = Dense(42, activation='tanh')(inputs)
    dense_2 = Dense(25, activation='tanh')(dense_1)
    dense_3 = Dense(5, activation='tanh')(dense_2)
    dropout = Dropout(0.2)(dense_3)
    outputs = Dense(2,activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        #loss='binary_crossentropy',
        optimizer=optimizers,
        metrics=['mse'],
    )

    return model

In [485]:
final_model = my_model()

In [ ]:
final_model.fit(fff_train_x,np.array(fff_y),epochs= 30 ,validation_split= 0.01)

In [ ]:
final_model.save("final_model_new.bin")